# Task4 基于深度学习的文本分类——FastText
在上一章节，介绍几种文本表示方法：  
One-hot  
Bag of Words  
N-gram  
TF-IDF  

但上述方法都或多或少存在一定的问题：转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计。  

与这些表示方法不同，深度学习也可以用于文本表示，还可以将其映射到一个低纬空间。其中比较典型的例子有：FastText、Word2Vec和Bert。在本章我们将介绍FastText，将在后面的内容介绍Word2Vec和Bert。

FastText是一种典型的深度学习词向量的表示方法，它非常简单通过Embedding层将单词映射到稠密空间，然后将句子中所有的单词在Embedding空间中进行平均，进而完成分类操作。  
所以FastText是一个三层的神经网络，**输入层、隐含层和输出层**。
![](https://camo.githubusercontent.com/4e01004146c81db5ee15df1b373374b3ff145bfa/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303731343230343835363538392e706e67)  
它的优点非常明显，在文本分类任务中，FastText（浅层网络）往往能取得**和深度网络相媲美的精度**，却在**训练时间上比深度网络快许多数量级**。在标准的多核CPU上， 能够训练10亿词级别语料库的词向量在10分钟之内，能够分类有着30万多类别的50多万句子在1分钟之内。

## 三、基于FastText的文本分类
* fasttext安装  
参考：https://blog.csdn.net/weixin_45116604/article/details/107587032

* 分类模型  

In [2]:
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

import fasttext
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs")

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))
# 0.82

0.8217235312185116


此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加，5w条训练样本时，验证集得分可以到0.89-0.90左右。